<a href="https://colab.research.google.com/github/mmopenat/Image_processing/blob/main/runtime.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

# Load the data with manual splitting
file_path = '/content/GERM_data.csv'
germ_data = pd.read_csv(file_path, delimiter='\t', engine='python')

# Manually split the single column into multiple columns using tab separator
germ_data = germ_data.iloc[:, 0].str.split('\t', expand=True)

# Assign meaningful column names based on your provided column description
germ_data.columns = ['Country', 'REGION_1F2A', 'sitename_2ED0', 'ATOLL_ID_7C7A', 'devicename_2D2E',
                     'devicetype_ACF9', 'deviceIpAddress_AC55', 'deviceport_899E', 'LastResponse_8A12',
                     'Online_4B65', 'Online_Summary_93D5', 'Gen_Engine_Starts_199A',
                     'Gen_Engine_Runtime_Hrs_B29E', 'Gen_Fuel_Level_F6F2', 'Gen_Bat_Voltage_8DB4',
                     'Gen_Alt_Voltage_2338']

# Clean the data by removing extra quotes and whitespace
germ_data_cleaned = germ_data.apply(lambda x: x.str.replace('"', '').str.strip() if x.dtype == "object" else x)

# Convert the relevant runtime column to numeric for calculation
germ_data_cleaned['Gen_Engine_Runtime_Hrs_B29E'] = pd.to_numeric(germ_data_cleaned['Gen_Engine_Runtime_Hrs_B29E'], errors='coerce')

# Group the data by region and generator (sitename) and sum the runtime
runtime_per_region_generator_cleaned = germ_data_cleaned.groupby(['REGION_1F2A', 'sitename_2ED0'])['Gen_Engine_Runtime_Hrs_B29E'].sum()

# Save the result to an Excel file in the current working directory
output_file_path = 'runtime_per_region_generator.xlsx'
runtime_per_region_generator_cleaned.to_excel(output_file_path, sheet_name='Runtime_per_Region_Generator')

# Provide the file path for download
output_file_path



'runtime_per_region_generator.xlsx'

In [ ]:
import pandas as pd

# Load the data with manual splitting
file_path = '/content/GERM_data.csv'
germ_data = pd.read_csv(file_path, delimiter='\t', engine='python')

# Manually split the single column into multiple columns using tab separator
germ_data = germ_data.iloc[:, 0].str.split('\t', expand=True)

# Assign meaningful column names based on your provided column description
germ_data.columns = ['Country', 'REGION_1F2A', 'sitename_2ED0', 'ATOLL_ID_7C7A', 'devicename_2D2E',
                     'devicetype_ACF9', 'deviceIpAddress_AC55', 'deviceport_899E', 'LastResponse_8A12',
                     'Online_4B65', 'Online_Summary_93D5', 'Gen_Engine_Starts_199A',
                     'Gen_Engine_Runtime_Hrs_B29E', 'Gen_Fuel_Level_F6F2', 'Gen_Bat_Voltage_8DB4',
                     'Gen_Alt_Voltage_2338']

# Clean the data by removing extra quotes and whitespace
germ_data_cleaned = germ_data.apply(lambda x: x.str.replace('"', '').str.strip() if x.dtype == "object" else x)

# Convert the relevant runtime column to numeric for calculation
germ_data_cleaned['Gen_Engine_Runtime_Hrs_B29E'] = pd.to_numeric(germ_data_cleaned['Gen_Engine_Runtime_Hrs_B29E'], errors='coerce')

# Convert the 'LastResponse_8A12' column to datetime format for proper date handling
germ_data_cleaned['LastResponse_8A12'] = pd.to_datetime(germ_data_cleaned['LastResponse_8A12'], errors='coerce')

# Group the data by region and generator (sitename) and get the sum of runtime and the most recent response date
runtime_per_region_generator_dates = germ_data_cleaned.groupby(['REGION_1F2A', 'sitename_2ED0']).agg({
    'Gen_Engine_Runtime_Hrs_B29E': 'sum',
    'LastResponse_8A12': 'max'  # Get the most recent date for each generator
})

# Save the result to an Excel file in the current working directory
output_file_path = 'runtime_per_region_generator_with_dates.xlsx'
runtime_per_region_generator_dates.to_excel(output_file_path, sheet_name='Runtime_per_Region_Generator')

# Provide the file path for download
output_file_path


'runtime_per_region_generator_with_dates.xlsx'

In [ ]:
import pandas as pd

# Load the data with manual splitting
file_path = '/content/GERM_data.csv'
germ_data = pd.read_csv(file_path, delimiter='\t', engine='python')

# Manually split the single column into multiple columns using tab separator
germ_data = germ_data.iloc[:, 0].str.split('\t', expand=True)

# Assign meaningful column names based on your provided column description
germ_data.columns = ['Country', 'REGION_1F2A', 'sitename_2ED0', 'ATOLL_ID_7C7A', 'devicename_2D2E',
                     'devicetype_ACF9', 'deviceIpAddress_AC55', 'deviceport_899E', 'LastResponse_8A12',
                     'Online_4B65', 'Online_Summary_93D5', 'Gen_Engine_Starts_199A',
                     'Gen_Engine_Runtime_Hrs_B29E', 'Gen_Fuel_Level_F6F2', 'Gen_Bat_Voltage_8DB4',
                     'Gen_Alt_Voltage_2338']

# Clean the data by removing extra quotes and whitespace
germ_data_cleaned = germ_data.apply(lambda x: x.str.replace('"', '').str.strip() if x.dtype == "object" else x)

# Convert the 'Gen_Engine_Runtime_Hrs_B29E' to numeric
germ_data_cleaned['Gen_Engine_Runtime_Hrs_B29E'] = pd.to_numeric(germ_data_cleaned['Gen_Engine_Runtime_Hrs_B29E'], errors='coerce')

# Convert the 'LastResponse_8A12' to datetime format for proper date handling
germ_data_cleaned['LastResponse_8A12'] = pd.to_datetime(germ_data_cleaned['LastResponse_8A12'], errors='coerce')

# Sort the data by generator and date
germ_data_cleaned = germ_data_cleaned.sort_values(by=['sitename_2ED0', 'LastResponse_8A12'])

# Group the data by sitename (generator) and calculate the difference in runtime for consecutive rows (this will give daily runtime)
germ_data_cleaned['Daily_Runtime_Hrs'] = germ_data_cleaned.groupby('sitename_2ED0')['Gen_Engine_Runtime_Hrs_B29E'].diff()

# Drop any rows without valid runtime differences (e.g., the first row for each generator)
germ_data_cleaned = germ_data_cleaned.dropna(subset=['Daily_Runtime_Hrs'])

# Now group by both generator and date to calculate total runtime per day
germ_data_cleaned['date'] = germ_data_cleaned['LastResponse_8A12'].dt.date
runtime_per_day = germ_data_cleaned.groupby(['sitename_2ED0', 'date'])['Daily_Runtime_Hrs'].sum()

# Save the result to an Excel file
output_file_path = 'runtime_per_day.xlsx'
runtime_per_day.to_excel(output_file_path, sheet_name='Runtime_per_Day')

# Provide the file path for download
output_file_path


'runtime_per_day.xlsx'